In [2]:
import pydot_ng as pydot
import matplotlib.pyplot as plt 
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
import keras
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, Bidirectional, GRU
from keras.models import Sequential
from utils import *
from keras.metrics import binary_accuracy
from keras.callbacks import *
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.models import Model
from keras.layers.convolutional import MaxPooling1D, Conv1D
from keras.layers import Flatten
from keras.callbacks import *
from keras.utils import plot_model  
from keras.models import load_model

In [3]:
from utils import *
x_train, y_train, x_test, y_test = load_ASTD()

In [4]:
print(len(y_train))
print(len(y_test))

962
318


count     962
unique      2
top         0
freq      481
Name: sentiment, dtype: object

In [11]:
embedding_dim = 300
max_num_words = 10000
max_sequence_length = 30
embedding_path = './data/embeddings/wiki.ar.vec'
embedding_path = './data/embeddings/wiki_corpus_tweets_xtrain_word2vec_size300_skip_gram_wv_iter=10'
gensim = True
print('ASTD dataset')
x_train, y_train, x_test, y_test = load_ASTD()
print('Tokenizing texts')
x_train, x_test, word_index = prepare_tokenized_data((x_train, x_test), max_num_words, max_sequence_length)
print('Finished tokenizing texts')
print('-' * 80)
embedding_matrix = load_embedding_matrix(embedding_path, word_index, embedding_dim,gensim=gensim)

ASTD dataset
Tokenizing texts
Saved tokenizer.pkl
Found 7277 unique 1-gram tokens.
Min sequence length: 0
Average sequence length: 14
Max sequence length: 29
Finished tokenizing texts
--------------------------------------------------------------------------------
Load embedding model.
Numbers of words in embedding model : 667062
Preparing embedding matrix.
Embeding matrix size : 7278
Saved embedding matrix


In [ ]:
def prepare_tokenized_data(data, max_num_words, max_sequence_length):
    train, test = data
    if not os.path.exists('data/tokenizer.pkl'):
        tokenizer = Tokenizer(num_words=max_num_words)
        tokenizer.fit_on_texts(train)

        with open('data/tokenizer.pkl', 'wb') as f:
            pickle.dump(tokenizer, f)

        print('Saved tokenizer.pkl')
    else:
        with open('data/tokenizer.pkl', 'rb') as f:
            tokenizer = pickle.load(f)
            print('Loaded tokenizer.pkl')

    sequences_train = tokenizer.texts_to_sequences(train)
    sequences_test = tokenizer.texts_to_sequences(test)
    word_index = tokenizer.word_index
    print('Found %s unique 1-gram tokens.' % len(word_index))
    print('Min sequence length: {}'.format(np.min(list(map(len, sequences_train)))))
    print('Average sequence length: {}'.format(np.mean(list(map(len, sequences_train)), dtype=int)))
    print('Max sequence length: {}'.format(np.max(list(map(len, sequences_train)))))
    train = pad_sequences(sequences_train, maxlen=max_sequence_length)
    test = pad_sequences(sequences_test, maxlen=max_sequence_length)
    return (train, test, word_index)

# Our dataset

In [2]:
embedding_dim = 300
max_num_words = 100000
max_sequence_length = 30
validation_split = 0.3
embedding_path = './data/embeddings/wiki.ar.vec'
embedding_path = './data/embeddings/wiki_corpus_tweets_xtrain_word2vec_size300_skip_gram_wv_iter=10'

full_dataset = True
x_train, y_train, x_test, y_test, word_index = prepare_data(max_num_words,max_sequence_length,validation_split,full_dataset)
embedding_matrix = load_embedding_matrix(embedding_path, word_index, embedding_dim)

Number of positive tweets :  100800 

Number of negative tweets :  99322 

indices : [     0      1      2 ..., 200119 200120 200121]
Test split : 30.0 %
Number of samples : 200122
Train size : 140086
Test size : 60036
Tokenizing texts
Saved tokenizer.pkl
Found 112611 unique 1-gram tokens.
Min sequence length: 2
Average sequence length: 13
Max sequence length: 49
Finished tokenizing texts
--------------------------------------------------------------------------------
Load embedding model.
Numbers of words in embedding model : 667062
Preparing embedding matrix.
Embeding matrix size : 112612
Saved embedding matrix


In [10]:
y_test.value_counts()
from keras import backend as K
K.clear_session()

# Bidirectionelle GRU

In [12]:
K.clear_session()
model_bi_gru = Sequential()
model_bi_gru.add(Embedding(len(word_index) + 1,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=False))
model_bi_gru.add(Bidirectional(GRU(300,activation='linear')))
model_bi_gru.add(Activation('relu'))
model_bi_gru.add(Dropout(0.5))
model_bi_gru.add(Dense(64,activation='relu'))
model_bi_gru.add(Dense(1, activation='sigmoid'))
# Compile model
model_bi_gru.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# checkpoint
#bi_gru_filepath="./saved_models/ASTD.Bi.GRU.model-{val_acc:.2f}.hdf5"
bi_gru_filepath="./saved_models/DATA.Bi.GRU.model-{val_acc:.2f}.hdf5"
#checkpoint = ModelCheckpoint(bi_gru_filepath, monitor='val_acc', verbose=2, save_best_only=True, mode='max')
tbCallBack = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=False,embeddings_freq = 1 ,
                                    embeddings_layer_names = None ,
                                    embeddings_metadata = None)
#history2 = model.fit(x_train,y_train,batch_size=100,epochs=10,validation_data=(x_test,y_test),verbose=1,callbacks=[checkpoint])

In [13]:
history_bi_gru = model_bi_gru.fit(x_train,y_train,batch_size=100,epochs=3,
                                  validation_data=(x_test,y_test),verbose=1,callbacks=[tbCallBack])

Train on 962 samples, validate on 318 samples
Epoch 1/3
962/962 [==============================] - 7s - loss: 0.6812 - acc: 0.5696 - val_loss: 0.6518 - val_acc: 0.6981
Epoch 2/3
962/962 [==============================] - 6s - loss: 0.5670 - acc: 0.7339 - val_loss: 0.5430 - val_acc: 0.7327
Epoch 3/3
962/962 [==============================] - 7s - loss: 0.4455 - acc: 0.7890 - val_loss: 0.5129 - val_acc: 0.7547


In [6]:
model_bi_gru = load_model('./data/data_model/DATA.Bi.GRU.model-0.88.hdf5')
y_pred = model_bi_gru.predict_classes(x_test)
evaluate(y_test,y_pred)

60032/60036 [============================>.] - ETA: 0sAccuracy score :  0.88 

Precision score :  0.87 

Recall score:  0.90 

F1 score :  0.89 



In [ ]:
history =history_bi_gru
print(history.history.keys())  

plt.figure(1)  

# summarize history for accuracy  
plt.subplot(211)  
plt.plot(history.history['acc'])  
plt.plot(history.history['val_acc'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='lower right')  
# summarize history for loss  


# train loss

In [ ]:
plt.subplot(111)  
plt.plot(history.history['loss'])  
#plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['base d\'entrainement', 'base de test'], loc='upper right')  
plt.show()  

# val_loss 

In [ ]:
plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='lower right')  

plt.show()  